In [31]:
# === Cell 1. 공통 설정 & 임포트 (전처리+학습 통합) ===
import os, json, warnings, unicodedata
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, f1_score
)
from sklearn.calibration import CalibratedClassifierCV
from joblib import dump

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

warnings.filterwarnings("ignore")

# ---------- 표시 옵션 ----------
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# ---------- 경로/상수 ----------
DATASETS_DIR  = Path("datasets") / "gender"                 # ✅ ./datasets/gender
INPUT_PATH    = "datasets/processed_user_behavior.joined.csv"
OUTPUT_DIR    = DATASETS_DIR                                # ✅ 산출물 저장 경로
ARTIFACT_DIR  = Path("models") / "gender"                   # ./models/gender

# 폴더 생성(이미 있어도 OK)
DATASETS_DIR.mkdir(parents=True, exist_ok=True)
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

# 프리로그인 이벤트가 2개 이상인 세션만 사용
MIN_PRELOGIN_EVENTS = 2

# 카테고리 5개만 사용 (OTHER 제거). 필요 시 실제 5개로 교체
KNOWN_CATS = ["Books", "Electronics", "Gaming", "Home", "Fashion"]

RANDOM_STATE = 42
PRODUCTION_THRESHOLD = 0.444   # 운영 임계값 (고정)

def norm(s: pd.Series) -> pd.Series:
    """공백 제거 + 유니코드 NFKC 정규화(NA 안전)"""
    s = s.astype("string").str.strip()
    return s.apply(lambda x: unicodedata.normalize("NFKC", x) if pd.notna(x) else x)


In [32]:
# === Cell 2. 원본 로드 & 정렬 & 프리로그인 구간 추출 ===
df = pd.read_csv(INPUT_PATH, low_memory=False)

# timestamp 파싱 & 정렬
df["ts"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.dropna(subset=["session_id", "ts"]).copy()

sort_cols = ["session_id", "ts"]
if "event_id" in df.columns:
    sort_cols.append("event_id")  # 있으면 정렬 안정성 ↑
df = df.sort_values(sort_cols).reset_index(drop=True)

print("원본 행 수:", len(df))

# 로그인 여부 판단
uid_str  = df["user_id"].astype("string").str.strip()
anon_like = {"", "0", "-1", "None", "none", "NULL", "null", "NaN", "nan"}
has_uid = uid_str.notna() & ~uid_str.isin(anon_like)

# 세션 내에서 한 번이라도 user_id가 등장하면 그 시점 이후는 로그인 이후
appeared = has_uid.groupby(df["session_id"]).cummax()
pre = df.loc[~appeared].copy()

# 프리로그인 이벤트가 2개 이상인 세션만 유지
prelogin_counts = pre.groupby("session_id").size()
valid_sessions = prelogin_counts.index[prelogin_counts >= MIN_PRELOGIN_EVENTS]
pre = pre[pre["session_id"].isin(valid_sessions)].copy()

print("프리로그인 필터 후 행 수:", len(pre))
print("유지된 프리로그인 세션 수:", pre["session_id"].nunique())


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/processed_user_behavior.joined.csv'

In [33]:
# === Cell 3. 라벨(M/F) 생성(세션 전체) + 프리로그인 수치 집계 ===
# 세션 전체에서 첫 유효 gender(M/F)
df_gender = df.copy()
df_gender["gender_norm"] = (
    df_gender["gender"]
      .astype("string").str.strip().str.upper()
      .replace({"FEMALE":"F", "MALE":"M"})
)
lab_full = (
    df_gender[df_gender["gender_norm"].isin(["M","F"])]
      .groupby("session_id")["gender_norm"].agg(lambda s: s.iloc[0])
)

# 프리로그인 수치 피처 집계
agg_num = pre.groupby("session_id").agg(
    n_events=("session_id", "size"),
    search_count_sum=("search_count", "sum"),
    cart_item_count_sum=("cart_item_count", "sum"),
    page_depth_mean=("page_depth", "mean"),
    last_elapsed_mean=("last_action_elapsed", "mean"),
    unique_pages=("current_state", "nunique"),
    unique_categories=("resolved_category", "nunique"),
).fillna(0.0)

# 세션 시작 시간 파생
first_ts = pre.groupby("session_id")["ts"].min()
agg_num["start_hour"] = first_ts.dt.hour
agg_num["start_weekday"] = first_ts.dt.weekday

# 라벨과 프리로그인 피처의 교집합 세션만 사용
keep_sessions = agg_num.index.intersection(lab_full.index)
X_num = agg_num.loc[keep_sessions].copy()
y = lab_full.loc[keep_sessions].rename("gender").copy()

print("교집합 세션 수:", len(keep_sessions))
print("라벨 분포:\n", y.value_counts(dropna=False))


교집합 세션 수: 649
라벨 분포:
 gender
F    344
M    305
Name: count, dtype: Int64


In [34]:
# === Cell 4. 카테고리 5개(OTHER 제거) 카운트/비율/로그 파생 ===
# KNOWN_CATS 정규화
KNOWN_CATS_NORM = [unicodedata.normalize("NFKC", c.strip()) for c in KNOWN_CATS]
norm_to_orig = {unicodedata.normalize("NFKC", c.strip()): c for c in KNOWN_CATS}

# 프리로그인 구간 카테고리 정규화
pre_cat_norm = norm(pre["resolved_category"])

# 5개 목록에 포함된 이벤트만 집계(OTHER 제거)
mask = pre_cat_norm.isin(KNOWN_CATS_NORM)
pre_kept = pre[mask].copy()
pre_kept["cat_norm"] = pre_cat_norm[mask].values
pre_kept["one"] = 1

# 세션 × 카테고리 카운트
cat_cnt = pre_kept.pivot_table(
    index="session_id",
    columns="cat_norm",
    values="one",
    aggfunc="sum",
    fill_value=0
)

# 컬럼을 정확히 5개로 강제(없으면 0) + 원래 이름으로 되돌리기
cat_cnt = cat_cnt.reindex(columns=KNOWN_CATS_NORM, fill_value=0)
cat_cnt.columns = [norm_to_orig[c] for c in cat_cnt.columns]

# prefix & 정렬(교집합 세션만)
cat_cnt.columns = [f"cat_cnt::{c}" for c in cat_cnt.columns]
cat_cnt = cat_cnt.reindex(X_num.index).fillna(0).astype(int)

# 이후 파생에 사용할 원 컬럼명 목록
cat_cnt_cols = list(cat_cnt.columns)

# prop = count / n_events (세션 길이 보정)
cat_prop = cat_cnt.div(X_num["n_events"].replace(0, 1), axis=0)
cat_prop.columns = [c.replace("cat_cnt::", "cat_prop::") for c in cat_cnt_cols]

# log1p(count) — 선형모델용 강도 완화
cat_log = np.log1p(cat_cnt)
cat_log.columns = [c.replace("cat_cnt::", "cat_log::") for c in cat_cnt_cols]

print("카테고리 카운트 컬럼:", list(cat_cnt.columns))
print("카테고리 파생 컬럼(예시 3개씩):", list(cat_prop.columns[:3]) + list(cat_log.columns[:3]))


카테고리 카운트 컬럼: ['cat_cnt::Books', 'cat_cnt::Electronics', 'cat_cnt::Gaming', 'cat_cnt::Home', 'cat_cnt::Fashion']
카테고리 파생 컬럼(예시 3개씩): ['cat_prop::Books', 'cat_prop::Electronics', 'cat_prop::Gaming', 'cat_log::Books', 'cat_log::Electronics', 'cat_log::Gaming']


In [35]:
# === Cell 5. 최종 결합(X) & 저장(선택) ===
# 피처 결합
X = (
    X_num
    .join(cat_cnt, how="left")
    .join(cat_prop, how="left")
    .join(cat_log, how="left")
).fillna(0)

# 라벨 결합 및 미리보기
X_ = X.copy()
if X_.index.name != "session_id":
    X_.index.name = "session_id"

dataset = X_.join(y).reset_index()

print("dataset shape:", dataset.shape)
print(dataset.head(10))

# (선택) 중간 산출물 저장
FEATURES_CSV = f"{OUTPUT_DIR}/prelogin_gender_features.csv"
FEATURES_PARQUET = f"{OUTPUT_DIR}/prelogin_gender_features.parquet"
X_only_csv = f"{OUTPUT_DIR}/prelogin_features_only.csv"
X_only_parquet = f"{OUTPUT_DIR}/prelogin_features_only.parquet"
y_only_csv = f"{OUTPUT_DIR}/prelogin_labels_only.csv"
y_only_parquet = f"{OUTPUT_DIR}/prelogin_labels_only.parquet"

dataset.to_csv(FEATURES_CSV, index=False)
dataset.to_parquet(FEATURES_PARQUET, index=False)
X_.reset_index().to_csv(X_only_csv, index=False)
X_.reset_index().to_parquet(X_only_parquet, index=False)
y.to_frame("gender").reset_index().to_csv(y_only_csv, index=False)
y.to_frame("gender").reset_index().to_parquet(y_only_parquet, index=False)

print(f"\nSaved: {FEATURES_CSV}")
print(f"Saved: {FEATURES_PARQUET}")
print("Saved (X only):", X_only_csv, "|", X_only_parquet)
print("Saved (y only):", y_only_csv, "|", y_only_parquet)


dataset shape: (649, 26)
                             session_id  n_events  search_count_sum  cart_item_count_sum  page_depth_mean  last_elapsed_mean  unique_pages  unique_categories  start_hour  start_weekday  \
0  0022505a-eba9-4865-994b-2253b3c4444c         4                 0                    0              2.5           1.000000             4                  1          13              2   
1  00ba3fe8-1d64-4e4d-9803-1871df07f69b         8                 3                    0              4.5           1.250000             7                  1           6              4   
2  01791a54-64a9-4336-93a6-81853d8d4ee8        13                 8                    0              7.0           1.153846             9                  2          13              2   
3  017c1695-2c78-4a45-83a5-e11c4ae061b8         4                 0                    0              2.5           2.000000             4                  0          14              2   
4  0185ccfb-814a-4ad0-82d8-06877765

In [36]:
# === Cell 6. X/y 분리, 학습/검증 분할 ===
# gender 정규화
dataset["gender"] = (
    dataset["gender"].astype("string").str.strip().str.upper()
    .replace({"FEMALE":"F","MALE":"M"})
)

# feature/label 분리
non_feature = {"session_id","gender"}
feature_cols = [c for c in dataset.columns if c not in non_feature]
X = dataset[feature_cols].copy()
y_bin = dataset["gender"].map({"F":0, "M":1}).astype(int)

# 학습/검증 stratified split (8:2)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, valid_idx = next(sss.split(X, y_bin))
X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
y_train, y_valid = y_bin.iloc[train_idx], y_bin.iloc[valid_idx]

print("Train:", X_train.shape, " Valid:", X_valid.shape)
print("Label dist (train):\n", y_train.value_counts(normalize=True).rename({0:"F",1:"M"}))
print("Label dist (valid):\n", y_valid.value_counts(normalize=True).rename({0:"F",1:"M"}))


Train: (519, 24)  Valid: (130, 24)
Label dist (train):
 gender
F    0.529865
M    0.470135
Name: proportion, dtype: float64
Label dist (valid):
 gender
F    0.530769
M    0.469231
Name: proportion, dtype: float64


In [37]:
# === Cell 7. 보조 함수: 임계값 튜닝 & 평가 유틸 ===
def tune_threshold(model, X_va, y_va, search=(0.2,0.8,61)):
    """Macro-F1 최대화 임계값 탐색 (predict_proba 사용 가능한 모델 전제)"""
    lo, hi, n = search
    ths = np.linspace(lo, hi, n)
    proba = model.predict_proba(X_va)[:,1]
    best_t, best_f1 = 0.5, -1
    for t in ths:
        preds = (proba >= t).astype(int)
        f1 = f1_score(y_va, preds, average="macro")
        if f1 > best_f1:
            best_t, best_f1 = float(t), float(f1)
    return best_t, best_f1, proba

def evaluate(model, X_va, y_va, threshold=0.5, name="Model"):
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X_va)[:,1]
        y_hat = (proba >= threshold).astype(int)
    else:
        y_hat = model.predict(X_va)
        proba = None

    print(f"\n=== {name} @ threshold={threshold:.3f} ===")
    print(classification_report(y_va, y_hat, target_names=["F","M"], digits=4))
    print("Confusion matrix [rows=true F,M | cols=pred F,M]:\n", confusion_matrix(y_va, y_hat))
    return y_hat, proba

def print_scores(name, y_true, proba, t):
    y_pred = (proba >= t).astype(int)
    macro_f1 = f1_score(y_true, y_pred, average="macro")
    acc = (y_true.values == y_pred).mean()
    print(f"[{name}] Macro-F1 : {macro_f1:.4f} / Accuracy : {acc:.4f}")


In [38]:
# === Cell 8. Logistic 하이퍼파라미터 튜닝 + 확률 보정 ===
CALIB_METHOD = "isotonic"  # or "sigmoid"
CALIB_CV = 5

log_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("clf", LogisticRegression(
        solver="saga", max_iter=5000, random_state=RANDOM_STATE
    ))
])

# penalty 별 그리드 (충돌 회피 위해 분리)
log_grid = [
    {
        "scaler": [StandardScaler(with_mean=True, with_std=True),
                   RobustScaler(with_centering=True, with_scaling=True)],
        "clf__penalty": ["l2"],
        "clf__C": [0.2, 0.5, 1.0, 2.0, 5.0],
        "clf__class_weight": [None, "balanced"],
    },
    {
        "scaler": [StandardScaler(with_mean=True, with_std=True),
                   RobustScaler(with_centering=True, with_scaling=True)],
        "clf__penalty": ["l1"],
        "clf__C": [0.2, 0.5, 1.0, 2.0, 5.0],
        "clf__class_weight": [None, "balanced"],
    },
    {
        "scaler": [StandardScaler(with_mean=True, with_std=True),
                   RobustScaler(with_centering=True, with_scaling=True)],
        "clf__penalty": ["elasticnet"],
        "clf__l1_ratio": [0.2, 0.5, 0.8],
        "clf__C": [0.2, 0.5, 1.0, 2.0, 5.0],
        "clf__class_weight": [None, "balanced"],
    },
]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

log_search = GridSearchCV(
    estimator=log_pipe,
    param_grid=log_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=0,
)
log_search.fit(X_train, y_train)
log_best = log_search.best_estimator_
print("Logistic best params:", log_search.best_params_, " | cv f1_macro:", round(log_search.best_score_, 4))

# ===== 확률 보정 (best 모델 기반) =====
try:
    cal_log = CalibratedClassifierCV(estimator=log_best, method=CALIB_METHOD, cv=CALIB_CV)
except TypeError:  # 구버전 호환
    cal_log = CalibratedClassifierCV(base_estimator=log_best, method=CALIB_METHOD, cv=CALIB_CV)

cal_log.fit(X_train, y_train)
print(f"[Calibration] done: method={CALIB_METHOD}, cv={CALIB_CV}")


Logistic best params: {'clf__C': 0.2, 'clf__class_weight': None, 'clf__l1_ratio': 0.5, 'clf__penalty': 'elasticnet', 'scaler': RobustScaler()}  | cv f1_macro: 0.5986
[Calibration] done: method=isotonic, cv=5


In [39]:
# === Cell 9. 임계값 0.444 평가 & 아티팩트 저장 ===
# 보정된 모델(cal_log) 기준으로 prod=0.444만 출력
yhat_prod, _ = evaluate(
    cal_log, X_valid, y_valid,
    threshold=PRODUCTION_THRESHOLD,
    name=f"LogisticCal (prod={PRODUCTION_THRESHOLD:.3f})"
)
proba_v = cal_log.predict_proba(X_valid)[:,1]
print_scores(f"LogisticCal (prod@{PRODUCTION_THRESHOLD:.3f})", y_valid, proba_v, PRODUCTION_THRESHOLD)

# ===== 아티팩트 저장 =====
dump(log_best, os.path.join(ARTIFACT_DIR, "logistic_best.joblib"))
dump(cal_log,  os.path.join(ARTIFACT_DIR, "logistic_calibrated.joblib"))

# JSON 직렬화 보조: 객체 → {name, params}
def _to_jsonable(v):
    if isinstance(v, (str, int, float, bool)) or v is None:
        return v
    if hasattr(v, "get_params"):
        return {"name": v.__class__.__name__, "params": v.get_params(deep=False)}
    return str(v)

best_params_jsonable = {k: _to_jsonable(v) for k, v in log_search.best_params_.items()}

with open(os.path.join(ARTIFACT_DIR, "logistic_meta.json"), "w", encoding="utf-8") as f:
    json.dump({
        "best_params": best_params_jsonable,
        "best_cv_f1_macro": float(log_search.best_score_),
        "production_threshold": float(PRODUCTION_THRESHOLD),
        "calibration": {"method": CALIB_METHOD, "cv": CALIB_CV}
    }, f, ensure_ascii=False, indent=2)

# 검증 예측 저장
pred_col = f"y_pred@{PRODUCTION_THRESHOLD:.3f}"
out_df = pd.DataFrame({
    "session_id": dataset.iloc[valid_idx]["session_id"].values if "session_id" in dataset.columns else np.arange(len(valid_idx)),
    "y_true": y_valid.values,
    pred_col: (proba_v >= PRODUCTION_THRESHOLD).astype(int),
    "proba_cal": proba_v
})
out_df.to_csv(os.path.join(ARTIFACT_DIR, "logistic_valid_predictions.csv"), index=False)

print("\n[Saved] logistic_best.joblib, logistic_calibrated.joblib, logistic_meta.json, logistic_valid_predictions.csv")
print(f"[Info] production_threshold={PRODUCTION_THRESHOLD:.3f}")



=== LogisticCal (prod=0.444) @ threshold=0.444 ===
              precision    recall  f1-score   support

           F     0.6562    0.6087    0.6316        69
           M     0.5909    0.6393    0.6142        61

    accuracy                         0.6231       130
   macro avg     0.6236    0.6240    0.6229       130
weighted avg     0.6256    0.6231    0.6234       130

Confusion matrix [rows=true F,M | cols=pred F,M]:
 [[42 27]
 [22 39]]
[LogisticCal (prod@0.444)] Macro-F1 : 0.6229 / Accuracy : 0.6231

[Saved] logistic_best.joblib, logistic_calibrated.joblib, logistic_meta.json, logistic_valid_predictions.csv
[Info] production_threshold=0.444
